In [2]:
from pipeline import *

(CVXPY) Jul 02 07:24:36 PM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Unrecognized new version of ortools (9.5.2237). Expected < 9.5.0.Please open a feature request on cvxpy to enable support for this version.')
(CVXPY) Jul 02 07:24:36 PM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Unrecognized new version of ortools (9.5.2237). Expected < 9.5.0.Please open a feature request on cvxpy to enable support for this version.')


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from catboost import CatBoostClassifier

np.random.seed(101)

In [4]:
df = pd.read_parquet(r'C:\Users\minht\Desktop\MLOps\data\phase-1\prob-1\raw_train.parquet')

In [5]:
df = df.sample(frac=1)

In [6]:
df.head()

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,label,feature14,feature15,feature16
9983,"Surveyor, mining",personal_care,12.79,34288,30.007619,-80.907059,62009,27.317256,-82.898443,53.190542,18,6,2,0,7.487897,0.410416,63.042741
98173,Public librarian,entertainment,15.68,72703,39.777782,-94.761666,89379,35.323181,-93.987307,45.535533,18,5,2,0,5.633389,0.661364,57.932782
128394,Quality manager,kids_pets,6.35,1431,38.206944,-68.907868,3074,42.107080,-72.331161,25.440734,19,6,2,0,6.389406,0.378868,70.520838
78015,"Buyer, industrial",entertainment,79.99,67211,33.362880,-97.035489,409656,38.113737,-96.721361,93.755429,5,2,2,0,4.708116,0.377655,68.584359
90550,Call centre manager,misc_pos,29.70,60490,46.417758,-87.201743,73374,41.028301,-88.491522,30.991881,23,5,1,0,7.217449,0.424339,54.966319


In [7]:
train = df.drop(columns=['label'])
label = df['label']

In [8]:
X_train, X_val, y_train, y_val = train_test_split(train, label, test_size=0.2, random_state=101)

In [9]:
cats = ['feature1', 'feature2']

In [10]:
from sklearn.pipeline import FeatureUnion, Pipeline

In [11]:
columns = ['feature{}'.format(i) for i in range(1, 17)]

In [12]:
columns = ['feature3', 'feature15', 'feature11', 'feature16', 'feature2',
       'feature10', 'feature13', 'feature1', 'feature12', 'feature7']

In [13]:
s = Pipeline(
    steps=(
        ('feature_select', FeatureSelector(columns)),
        ('woe', WOE(cats=cats, nums=[]))
    )
)

In [14]:
s.fit(X_train, y_train)

Pipeline(steps=[('feature_select',
                 FeatureSelector(_feature_names=['feature3', 'feature15',
                                                 'feature11', 'feature16',
                                                 'feature2', 'feature10',
                                                 'feature13', 'feature1',
                                                 'feature12', 'feature7'])),
                ('woe', WOE(cats=['feature1', 'feature2'], nums=[]))])

In [15]:
import pickle

with open(r'C:\Users\minht\Desktop\MLOps\src\phase1\models\pipeline.pkl', 'wb') as f:
    pickle.dump(s, f)

In [16]:
X_train = s.transform(X_train)
X_val = s.transform(X_val)

C:\Users\minht\Desktop\MLOps\notebooks\phase1\pipeline.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = self.res[col].transform(X[col], metric='woe')
C:\Users\minht\Desktop\MLOps\notebooks\phase1\pipeline.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = X[col].apply(self._round)
C:\Users\minht\Desktop\MLOps\notebooks\phase1\pipeline.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

In [17]:
model = CatBoostClassifier(
    iterations=2000,
    random_seed=101,
    learning_rate=0.02,
    eval_metric='AUC'
)

In [18]:
model.fit(X_train, y_train, eval_set=(X_val, y_val))

0:	test: 0.9845517	best: 0.9845517 (0)	total: 65ms	remaining: 2m 9s
1:	test: 0.9870096	best: 0.9870096 (1)	total: 75.5ms	remaining: 1m 15s
2:	test: 0.9903960	best: 0.9903960 (2)	total: 86ms	remaining: 57.3s
3:	test: 0.9915387	best: 0.9915387 (3)	total: 95.9ms	remaining: 47.9s
4:	test: 0.9928163	best: 0.9928163 (4)	total: 106ms	remaining: 42.1s
5:	test: 0.9929351	best: 0.9929351 (5)	total: 115ms	remaining: 38.2s
6:	test: 0.9930022	best: 0.9930022 (6)	total: 124ms	remaining: 35.4s
7:	test: 0.9930875	best: 0.9930875 (7)	total: 134ms	remaining: 33.3s
8:	test: 0.9928070	best: 0.9930875 (7)	total: 143ms	remaining: 31.6s
9:	test: 0.9931131	best: 0.9931131 (9)	total: 152ms	remaining: 30.2s
10:	test: 0.9929959	best: 0.9931131 (9)	total: 161ms	remaining: 29.1s
11:	test: 0.9928759	best: 0.9931131 (9)	total: 170ms	remaining: 28.2s
12:	test: 0.9930229	best: 0.9931131 (9)	total: 180ms	remaining: 27.5s
13:	test: 0.9932495	best: 0.9932495 (13)	total: 189ms	remaining: 26.8s
14:	test: 0.9932133	best: 0.

127:	test: 0.9979070	best: 0.9979070 (127)	total: 1.28s	remaining: 18.7s
128:	test: 0.9979257	best: 0.9979257 (128)	total: 1.29s	remaining: 18.7s
129:	test: 0.9979544	best: 0.9979544 (129)	total: 1.3s	remaining: 18.7s
130:	test: 0.9979665	best: 0.9979665 (130)	total: 1.31s	remaining: 18.7s
131:	test: 0.9979679	best: 0.9979679 (131)	total: 1.32s	remaining: 18.7s
132:	test: 0.9979801	best: 0.9979801 (132)	total: 1.33s	remaining: 18.6s
133:	test: 0.9979815	best: 0.9979815 (133)	total: 1.34s	remaining: 18.6s
134:	test: 0.9979962	best: 0.9979962 (134)	total: 1.35s	remaining: 18.6s
135:	test: 0.9980005	best: 0.9980005 (135)	total: 1.36s	remaining: 18.6s
136:	test: 0.9980071	best: 0.9980071 (136)	total: 1.37s	remaining: 18.6s
137:	test: 0.9980136	best: 0.9980136 (137)	total: 1.38s	remaining: 18.6s
138:	test: 0.9980132	best: 0.9980136 (137)	total: 1.39s	remaining: 18.6s
139:	test: 0.9980395	best: 0.9980395 (139)	total: 1.4s	remaining: 18.5s
140:	test: 0.9980406	best: 0.9980406 (140)	total: 1.4

253:	test: 0.9985391	best: 0.9985391 (253)	total: 2.5s	remaining: 17.2s
254:	test: 0.9985406	best: 0.9985406 (254)	total: 2.52s	remaining: 17.2s
255:	test: 0.9985421	best: 0.9985421 (255)	total: 2.52s	remaining: 17.2s
256:	test: 0.9985460	best: 0.9985460 (256)	total: 2.54s	remaining: 17.2s
257:	test: 0.9985482	best: 0.9985482 (257)	total: 2.54s	remaining: 17.2s
258:	test: 0.9985495	best: 0.9985495 (258)	total: 2.55s	remaining: 17.2s
259:	test: 0.9985507	best: 0.9985507 (259)	total: 2.56s	remaining: 17.2s
260:	test: 0.9985540	best: 0.9985540 (260)	total: 2.57s	remaining: 17.1s
261:	test: 0.9985600	best: 0.9985600 (261)	total: 2.58s	remaining: 17.1s
262:	test: 0.9985637	best: 0.9985637 (262)	total: 2.59s	remaining: 17.1s
263:	test: 0.9985645	best: 0.9985645 (263)	total: 2.6s	remaining: 17.1s
264:	test: 0.9985657	best: 0.9985657 (264)	total: 2.61s	remaining: 17.1s
265:	test: 0.9985679	best: 0.9985679 (265)	total: 2.62s	remaining: 17.1s
266:	test: 0.9985679	best: 0.9985679 (266)	total: 2.6

375:	test: 0.9987855	best: 0.9987855 (375)	total: 3.72s	remaining: 16.1s
376:	test: 0.9987906	best: 0.9987906 (376)	total: 3.73s	remaining: 16s
377:	test: 0.9987919	best: 0.9987919 (377)	total: 3.74s	remaining: 16s
378:	test: 0.9987909	best: 0.9987919 (377)	total: 3.75s	remaining: 16s
379:	test: 0.9987895	best: 0.9987919 (377)	total: 3.76s	remaining: 16s
380:	test: 0.9987923	best: 0.9987923 (380)	total: 3.77s	remaining: 16s
381:	test: 0.9987931	best: 0.9987931 (381)	total: 3.78s	remaining: 16s
382:	test: 0.9987946	best: 0.9987946 (382)	total: 3.79s	remaining: 16s
383:	test: 0.9987962	best: 0.9987962 (383)	total: 3.8s	remaining: 16s
384:	test: 0.9988001	best: 0.9988001 (384)	total: 3.81s	remaining: 16s
385:	test: 0.9988006	best: 0.9988006 (385)	total: 3.82s	remaining: 16s
386:	test: 0.9988026	best: 0.9988026 (386)	total: 3.83s	remaining: 16s
387:	test: 0.9988031	best: 0.9988031 (387)	total: 3.84s	remaining: 15.9s
388:	test: 0.9988034	best: 0.9988034 (388)	total: 3.85s	remaining: 15.9s
3

500:	test: 0.9989110	best: 0.9989110 (500)	total: 4.94s	remaining: 14.8s
501:	test: 0.9989117	best: 0.9989117 (501)	total: 4.95s	remaining: 14.8s
502:	test: 0.9989101	best: 0.9989117 (501)	total: 4.96s	remaining: 14.8s
503:	test: 0.9989114	best: 0.9989117 (501)	total: 4.97s	remaining: 14.8s
504:	test: 0.9989128	best: 0.9989128 (504)	total: 4.98s	remaining: 14.7s
505:	test: 0.9989110	best: 0.9989128 (504)	total: 4.99s	remaining: 14.7s
506:	test: 0.9989109	best: 0.9989128 (504)	total: 5s	remaining: 14.7s
507:	test: 0.9989122	best: 0.9989128 (504)	total: 5.01s	remaining: 14.7s
508:	test: 0.9989124	best: 0.9989128 (504)	total: 5.02s	remaining: 14.7s
509:	test: 0.9989133	best: 0.9989133 (509)	total: 5.03s	remaining: 14.7s
510:	test: 0.9989147	best: 0.9989147 (510)	total: 5.04s	remaining: 14.7s
511:	test: 0.9989163	best: 0.9989163 (511)	total: 5.05s	remaining: 14.7s
512:	test: 0.9989159	best: 0.9989163 (511)	total: 5.06s	remaining: 14.7s
513:	test: 0.9989184	best: 0.9989184 (513)	total: 5.07

...

1384:	test: 0.9991705	best: 0.9991705 (1383)	total: 13.4s	remaining: 5.95s
1385:	test: 0.9991708	best: 0.9991708 (1385)	total: 13.4s	remaining: 5.94s
1386:	test: 0.9991704	best: 0.9991708 (1385)	total: 13.4s	remaining: 5.93s
1387:	test: 0.9991707	best: 0.9991708 (1385)	total: 13.4s	remaining: 5.92s
1388:	test: 0.9991705	best: 0.9991708 (1385)	total: 13.4s	remaining: 5.91s
1389:	test: 0.9991709	best: 0.9991709 (1389)	total: 13.4s	remaining: 5.9s
1390:	test: 0.9991711	best: 0.9991711 (1390)	total: 13.5s	remaining: 5.89s
1391:	test: 0.9991717	best: 0.9991717 (1391)	total: 13.5s	remaining: 5.88s
1392:	test: 0.9991719	best: 0.9991719 (1392)	total: 13.5s	remaining: 5.87s
1393:	test: 0.9991723	best: 0.9991723 (1393)	total: 13.5s	remaining: 5.86s
1394:	test: 0.9991718	best: 0.9991723 (1393)	total: 13.5s	remaining: 5.85s
1395:	test: 0.9991723	best: 0.9991723 (1393)	total: 13.5s	remaining: 5.84s
1396:	test: 0.9991719	best: 0.9991723 (1393)	total: 13.5s	remaining: 5.83s
1397:	test: 0.9991717	best

1508:	test: 0.9991694	best: 0.9991789 (1421)	total: 14.6s	remaining: 4.75s
1509:	test: 0.9991703	best: 0.9991789 (1421)	total: 14.6s	remaining: 4.74s
1510:	test: 0.9991705	best: 0.9991789 (1421)	total: 14.6s	remaining: 4.73s
1511:	test: 0.9991706	best: 0.9991789 (1421)	total: 14.6s	remaining: 4.72s
1512:	test: 0.9991718	best: 0.9991789 (1421)	total: 14.6s	remaining: 4.71s
1513:	test: 0.9991723	best: 0.9991789 (1421)	total: 14.6s	remaining: 4.7s
1514:	test: 0.9991726	best: 0.9991789 (1421)	total: 14.7s	remaining: 4.69s
1515:	test: 0.9991727	best: 0.9991789 (1421)	total: 14.7s	remaining: 4.68s
1516:	test: 0.9991722	best: 0.9991789 (1421)	total: 14.7s	remaining: 4.67s
1517:	test: 0.9991730	best: 0.9991789 (1421)	total: 14.7s	remaining: 4.66s
1518:	test: 0.9991731	best: 0.9991789 (1421)	total: 14.7s	remaining: 4.65s
1519:	test: 0.9991735	best: 0.9991789 (1421)	total: 14.7s	remaining: 4.64s
1520:	test: 0.9991742	best: 0.9991789 (1421)	total: 14.7s	remaining: 4.63s
1521:	test: 0.9991736	best

1629:	test: 0.9991859	best: 0.9991891 (1595)	total: 15.8s	remaining: 3.59s
1630:	test: 0.9991853	best: 0.9991891 (1595)	total: 15.8s	remaining: 3.58s
1631:	test: 0.9991852	best: 0.9991891 (1595)	total: 15.8s	remaining: 3.57s
1632:	test: 0.9991854	best: 0.9991891 (1595)	total: 15.8s	remaining: 3.56s
1633:	test: 0.9991852	best: 0.9991891 (1595)	total: 15.8s	remaining: 3.55s
1634:	test: 0.9991852	best: 0.9991891 (1595)	total: 15.9s	remaining: 3.54s
1635:	test: 0.9991853	best: 0.9991891 (1595)	total: 15.9s	remaining: 3.53s
1636:	test: 0.9991853	best: 0.9991891 (1595)	total: 15.9s	remaining: 3.52s
1637:	test: 0.9991851	best: 0.9991891 (1595)	total: 15.9s	remaining: 3.51s
1638:	test: 0.9991849	best: 0.9991891 (1595)	total: 15.9s	remaining: 3.5s
1639:	test: 0.9991852	best: 0.9991891 (1595)	total: 15.9s	remaining: 3.49s
1640:	test: 0.9991851	best: 0.9991891 (1595)	total: 15.9s	remaining: 3.48s
1641:	test: 0.9991850	best: 0.9991891 (1595)	total: 15.9s	remaining: 3.47s
1642:	test: 0.9991853	best

1754:	test: 0.9991889	best: 0.9991891 (1595)	total: 17s	remaining: 2.38s
1755:	test: 0.9991890	best: 0.9991891 (1595)	total: 17s	remaining: 2.37s
1756:	test: 0.9991889	best: 0.9991891 (1595)	total: 17.1s	remaining: 2.36s
1757:	test: 0.9991887	best: 0.9991891 (1595)	total: 17.1s	remaining: 2.35s
1758:	test: 0.9991885	best: 0.9991891 (1595)	total: 17.1s	remaining: 2.34s
1759:	test: 0.9991882	best: 0.9991891 (1595)	total: 17.1s	remaining: 2.33s
1760:	test: 0.9991876	best: 0.9991891 (1595)	total: 17.1s	remaining: 2.32s
1761:	test: 0.9991879	best: 0.9991891 (1595)	total: 17.1s	remaining: 2.31s
1762:	test: 0.9991881	best: 0.9991891 (1595)	total: 17.1s	remaining: 2.3s
1763:	test: 0.9991874	best: 0.9991891 (1595)	total: 17.1s	remaining: 2.29s
1764:	test: 0.9991875	best: 0.9991891 (1595)	total: 17.1s	remaining: 2.28s
1765:	test: 0.9991877	best: 0.9991891 (1595)	total: 17.1s	remaining: 2.27s
1766:	test: 0.9991885	best: 0.9991891 (1595)	total: 17.1s	remaining: 2.26s
1767:	test: 0.9991889	best: 0.

1880:	test: 0.9991900	best: 0.9991900 (1880)	total: 18.2s	remaining: 1.15s
1881:	test: 0.9991890	best: 0.9991900 (1880)	total: 18.3s	remaining: 1.14s
1882:	test: 0.9991894	best: 0.9991900 (1880)	total: 18.3s	remaining: 1.13s
1883:	test: 0.9991886	best: 0.9991900 (1880)	total: 18.3s	remaining: 1.13s
1884:	test: 0.9991893	best: 0.9991900 (1880)	total: 18.3s	remaining: 1.11s
1885:	test: 0.9991889	best: 0.9991900 (1880)	total: 18.3s	remaining: 1.1s
1886:	test: 0.9991890	best: 0.9991900 (1880)	total: 18.3s	remaining: 1.09s
1887:	test: 0.9991899	best: 0.9991900 (1880)	total: 18.3s	remaining: 1.09s
1888:	test: 0.9991896	best: 0.9991900 (1880)	total: 18.3s	remaining: 1.08s
1889:	test: 0.9991897	best: 0.9991900 (1880)	total: 18.3s	remaining: 1.07s
1890:	test: 0.9991887	best: 0.9991900 (1880)	total: 18.3s	remaining: 1.06s
1891:	test: 0.9991887	best: 0.9991900 (1880)	total: 18.3s	remaining: 1.05s
1892:	test: 0.9991889	best: 0.9991900 (1880)	total: 18.4s	remaining: 1.04s
1893:	test: 0.9991888	best

In [19]:
vl = pd.DataFrame([X_train.columns, model.get_feature_importance()]).T
vl.columns = ['Feature', 'Importance']
vl = vl.sort_values('Importance', ascending=False)
vl

,Feature,Importance
0,feature3,31.598868
1,feature15,18.921608
2,feature11,13.402734
4,feature2,9.479755
3,feature16,9.333613
5,feature10,6.639736
6,feature13,4.143005
8,feature12,2.53406
7,feature1,2.34793
9,feature7,1.598691


In [20]:
import pickle

with open(r'C:\Users\minht\Desktop\MLOps\src\phase1\models\problem1.pkl', 'wb') as f:
    pickle.dump(model, f)

In [27]:
cc = X_train.head(100)

In [28]:
%time
tmp = model.predict_proba(X_train)[:, 1]

CPU times: total: 0 ns
Wall time: 0 ns


In [29]:
tmp = model.predict_proba(X_train)[:, 1]
tmp = pd.DataFrame(tmp, columns=['Pro'])

In [32]:
tmp

,Pro,Range
0,0.000027,"(1.56e-05, 3.34e-05]"
1,0.000002,"(1.06e-06, 3.02e-06]"
2,0.000281,"(0.000199, 0.0007]"
3,0.000004,"(3.02e-06, 7.12e-06]"
4,0.000001,"(1.06e-06, 3.02e-06]"
...,...,...
107355,0.003213,"(0.0007, 0.00759]"
107356,0.000007,"(7.12e-06, 1.56e-05]"
107357,0.000158,"(7.62e-05, 0.000199]"
107358,0.000003,"(1.06e-06, 3.02e-06]"


In [40]:
tmp['Range'], _ = pd.qcut(tmp['Pro'], 10, retbins=True)

In [41]:
tmp['Range'].value_counts(normalize=True).sort_index()

(-0.0009999888, 1.06e-06]    0.1
(1.06e-06, 3.02e-06]         0.1
(3.02e-06, 7.12e-06]         0.1
(7.12e-06, 1.56e-05]         0.1
(1.56e-05, 3.34e-05]         0.1
(3.34e-05, 7.62e-05]         0.1
(7.62e-05, 0.000199]         0.1
(0.000199, 0.0007]           0.1
(0.0007, 0.00759]            0.1
(0.00759, 1.0]               0.1
Name: Range, dtype: float64

In [44]:
_

array([1.12262329e-08, 1.05848934e-06, 3.02380710e-06, 7.11921902e-06,
       1.56224858e-05, 3.33687196e-05, 7.61984550e-05, 1.99051388e-04,
       6.99545860e-04, 7.59180369e-03, 9.99999992e-01])